## Note : this code is Heavily inspired by UDACITY course. and WE will update it more. This is first Version

#### Note2 : this code is trained on turkish language. you can change it




In [2]:
# you may need to install these new libs. I used next line commands to install them in kaggle notebook
'''
all pip installs and commands : 
pip install pydub 
pip install python_speech_features
!apt install -y ffmpeg
'''

# ignore warnings. 
import warnings
warnings.filterwarnings('ignore')

## import libs

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import librosa
import pydub
import IPython.display as ipd
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile # reading the wavfile

# =========== < new import for approach 2 > ======================
from numpy.lib.stride_tricks import as_strided

from pydub import AudioSegment
from python_speech_features import mfcc
from mpl_toolkits.axes_grid1 import make_axes_locatable
import random
import tensorflow
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Lambda)
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint   
import os
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Activation
# =========== </new import for approach 2> ======================

# this is aparameter for audios.
SIGNAL_RATE = 16000
RNG_SEED = 123
NumberOfSamples = 1823 # write number of samples here. number of data you have.
#  this is the longest duration of an audio file.
longest_y_length = 16000 * 10

In [ ]:
#  this is the path to my files. change it to yours.
PATH  = '/kaggle/input/trsst/tr/'

In [ ]:
# to read an audio file you have many approaches. one of them is librosa library.
# librosa approach to read audio.
def read2(file):
#     sound = pydub.AudioSegment.from_file(file, format="mp3").set_frame_rate(8000)
    sound = pydub.AudioSegment.from_file(file, format="mp3")
#     librosa.load ahs a duration property. accept just this much audio.
    y, sr = librosa.load(file, sr=16000) # sound is 
    duration = librosa.core.get_duration(y=y , sr=sr )
#     y, sr = librosa.load(file, sr=8000)
    return sr, y, sound, duration

In [ ]:
def drop_col(df, cols):
    for col in cols:
        df.drop([col], axis=1, inplace=True)


def prepare_dataset(df_address):
    df =pd.read_csv(df_address, delimiter='\t', encoding='utf-8')
    drop_col(df, ['age', 'up_votes', 'client_id', 'accent', 'down_votes', 'gender'])
    duration_list = []
#     for path in list(df.path):
#         y, sr = librosa.load(PATH + 'clips/' +  path, sr=16000) # sound is 
#         duration = librosa.core.get_duration(y=y , sr=sr )
#         duration_list.append(duration)
#     df['durations'] = duration_list
    return df

you may be curious my csv file columns. We used Deep Speech Mozilla data. so Our csvs have these columns : 
*  age
* up_votes
* client_id
* accent
* down_votes
* gender
* sentece
* path

I just used 2 last columns

In [ ]:
# for test
paths_list= list(devv.path)
sr, y, sound, duration = read2(PATH + '/clips/' + paths_list[4])

# next 2 cells are for setting our characters. we could use tensorflow tokenizer but we wrote it here. to be honest tensorflow approach is better.

In [ ]:
# this code will say in this panda file and it's specific column. how many unique character do you have.
set(devv.sentence.apply(list).sum())

In [ ]:
# char_map.py
# set(trainn.sentence.apply(list).sum()) # this code says to us how many unque characters do we have in sentences

char_map_str = """
' 0
<SPACE> 1
a 2
b 3
c 4
ç 5
d 6
e 7
f 8
g 9
ğ 10
h 11
ı 12
i 13
j 14
k 15
l 16
m 17
n 18
o 19
ö 20
p 21
r 22
s 23
ş 24
t 25
u 26
ü 27
v 28
y 29
z 30
A 31
B 32
C 33
Ç 34
D 35
E 36
F 37
G 38
Ğ 39
H 40
I 41
İ 42
J 43
K 44
L 45
M 46
N 47
O 48
Ö 49
P 50
R 51
S 52
Ş 53
T 54
U 55
Ü 56
V 57
Y 58
Z 59
. 60
! 61
" 62
% 63
, 64
- 65
? 66
‘ 67
� 68
â 69
w 70 
W 71
q 72
Q 73
x 74
X 75
î 76
"""
# the "blank" character is mapped to 28

char_map = {}
index_map = {}
for line in char_map_str.strip().split('\n'):
    ch, index = line.split()
    char_map[ch] = int(index)
    index_map[int(index)+1] = ch
print(index_map[2])
index_map[2] = ' '
print(index_map[2])

# next Cells are for our important functions. 

In [ ]:
def calc_feat_dim(window, max_freq):
    return int(0.001 * window * max_freq) + 1

def conv_output_length(input_length, filter_size, border_mode, stride,
                       dilation=1):
    """ Compute the length of the output sequence after 1D convolution along
        time. Note that this function is in line with the function used in
        Convolution1D class from Keras.
    Params:
        input_length (int): Length of the input sequence.
        filter_size (int): Width of the convolution kernel.
        border_mode (str): Only support `same` or `valid`.
        stride (int): Stride size used in 1D convolution.
        dilation (int)
    """
    if input_length is None:
        return None
    assert border_mode in {'same', 'valid'}
    dilated_filter_size = filter_size + (filter_size - 1) * (dilation - 1)
    if border_mode == 'same':
        output_length = input_length
    elif border_mode == 'valid':
        output_length = input_length - dilated_filter_size + 1
    return (output_length + stride - 1) // stride


def spectrogram(samples, fft_length=256, sample_rate=2, hop_length=128):
    """
    Compute the spectrogram for a real signal.
    The parameters follow the naming convention of
    matplotlib.mlab.specgram
    Args:
        samples (1D array): input audio signal
        fft_length (int): number of elements in fft window
        sample_rate (scalar): sample rate
        hop_length (int): hop length (relative offset between neighboring
            fft windows).
    Returns: 
        x (2D array): spectrogram [frequency x time]
        freq (1D array): frequency of each row in x
    Note:
        This is a truncating computation e.g. if fft_length=10,
        hop_length=5 and the signal has 23 elements, then the
        last 3 elements will be truncated.
    """
    assert not np.iscomplexobj(samples), "Must not pass in complex numbers"

    window = np.hanning(fft_length)[:, None]
    window_norm = np.sum(window**2)

    # The scaling below follows the convention of
    # matplotlib.mlab.specgram which is the same as
    # matlabs specgram.
    scale = window_norm * sample_rate

    trunc = (len(samples) - fft_length) % hop_length
    x = samples[:len(samples) - trunc]

    # "stride trick" reshape to include overlap
    nshape = (fft_length, (len(x) - fft_length) // hop_length + 1)
    nstrides = (x.strides[0], x.strides[0] * hop_length)
    x = as_strided(x, shape=nshape, strides=nstrides)

    # window stride sanity check 
    assert np.all(x[:, 1] == samples[hop_length:(hop_length + fft_length)])

    # broadcast window, compute fft over columns and square mod 
    x = np.fft.rfft(x * window, axis=0)
    x = np.absolute(x)**2

    # scale, 2.0 for everything except dc and fft_length/2
    x[1:-1, :] *= (2.0 / scale)
    x[(0, -1), :] /= scale

    freqs = float(sample_rate) / fft_length * np.arange(x.shape[0])

    return x, freqs


def spectrogram_from_file(filename, step=10, window=20, max_freq=None,
                          eps=1e-14):
    """ Calculate the log of linear spectrogram from FFT energy
    Params:
        filename (str): Path to the audio file
        step (int): Step size in milliseconds between windows
        window (int): FFT window size in milliseconds
        max_freq (int): Only FFT bins corresponding to frequencies between
            [0, max_freq] are returned
        eps (float): Small value to ensure numerical stability (for ln(x))
    """
#     asssume we don't need next line ==>  PATH + 'clips/' + filename
    audio, sample_rate = librosa.load(PATH + 'clips/' + filename, sr=16000)
#     audio = padarray(audio, 160000)
    if audio.ndim >= 2:
        audio = np.mean(audio, 1)
    if max_freq is None:
        max_freq = sample_rate / 2
    if max_freq > sample_rate / 2:
        raise ValueError("max_freq must not be greater than half of "
                            " sample rate")
    if step > window:
        raise ValueError("step size must not be greater than window size")
    hop_length = int(0.001 * step * sample_rate)
    fft_length = int(0.001 * window * sample_rate)
    pxx, freqs = spectrogram(
        audio, fft_length=fft_length, sample_rate=sample_rate,
        hop_length=hop_length)
    ind = np.where(freqs <= max_freq)[0][-1] + 1
    return np.transpose(np.log(pxx[:ind, :] + eps))
#     next lines are replaced with prev lines
#     with soundfile.SoundFile(filename) as sound_file:
#         audio = sound_file.read(dtype='float32')
#         sample_rate = sound_file.samplerate
#         if audio.ndim >= 2:
#             audio = np.mean(audio, 1)
#         if max_freq is None:
#             max_freq = sample_rate / 2
#         if max_freq > sample_rate / 2:
#             raise ValueError("max_freq must not be greater than half of "
#                              " sample rate")
#         if step > window:
#             raise ValueError("step size must not be greater than window size")
#         hop_length = int(0.001 * step * sample_rate)
#         fft_length = int(0.001 * window * sample_rate)
#         pxx, freqs = spectrogram(
#             audio, fft_length=fft_length, sample_rate=sample_rate,
#             hop_length=hop_length)
#         ind = np.where(freqs <= max_freq)[0][-1] + 1
#     return np.transpose(np.log(pxx[:ind, :] + eps))


def text_to_int_sequence(text):
    """ Convert text to an integer sequence """
    int_sequence = []
#     text = text.lower()
#     print(text)
    for c in text:
#         print(c)
        if c == ' ':
            ch = char_map['<SPACE>']
        else:
            ch = char_map[c]
        int_sequence.append(ch)
    return int_sequence

def int_sequence_to_text(int_sequence):
    """ Convert an integer sequence to text """
    text = []
    for c in int_sequence:
        ch = index_map[c]
        text.append(ch)
    return text

# this is the way to pad all the y of audios to same size.
def padarray(A, size):
    t = size - len(A)
    return np.pad(A, pad_width=(0, t), mode='constant')

padded = padarray(y, 160000)
padded.shape

# OUR CUSTOM DATA GENERATOR

In [ ]:
class AudioGenerator():
    def __init__(self, step=10, window=20, max_freq=8000, mfcc_dim=26,
        minibatch_size=20, desc_file=None, spectrogram=True, max_duration=10.0, 
        sort_by_duration=False):
        """
        Params: 
            step (int): Step size in milliseconds between windows (for spectrogram ONLY)
            window (int): FFT window size in milliseconds (for spectrogram ONLY)
            max_freq (int): Only FFT bins corresponding to frequencies between
                [0, max_freq] are returned (for spectrogram ONLY)
            desc_file (str, optional): Path to a JSON-line file that contains
                labels and paths to the audio files. If this is None, then
                load metadata right away
        """

        self.feat_dim = calc_feat_dim(window, max_freq)
        self.mfcc_dim = mfcc_dim
        self.feats_mean = np.zeros((self.feat_dim,))
        self.feats_std = np.ones((self.feat_dim,))
        self.rng = random.Random(RNG_SEED)
        if desc_file is not None:
            self.load_metadata_from_desc_file(desc_file)
        self.step = step
        self.window = window
        self.max_freq = max_freq
        self.cur_train_index = 0
        self.cur_valid_index = 0
        self.cur_test_index = 0
        self.max_duration=max_duration
        self.minibatch_size = minibatch_size
        self.spectrogram = spectrogram
        self.sort_by_duration = sort_by_duration

    def get_batch(self, partition):
        """ Obtain a batch of train, validation, or test data
        """
        if partition == 'train':
            audio_paths = self.train_audio_paths
            cur_index = self.cur_train_index
            texts = self.train_texts
        elif partition == 'valid':
            audio_paths = self.valid_audio_paths
            cur_index = self.cur_valid_index
            texts = self.valid_texts
        elif partition == 'test':
            audio_paths = self.test_audio_paths
            cur_index = self.test_valid_index
            texts = self.test_texts
        else:
            raise Exception("Invalid partition. "
                "Must be train/validation")

        features = [self.normalize(self.featurize(a)) for a in 
            audio_paths[cur_index:cur_index+self.minibatch_size]]
        
        # calculate necessary sizes
#         calculate maximum length from spectogram features
        max_length = max([features[i].shape[0] 
            for i in range(0, self.minibatch_size)])
        
#     calculate miximum length of sentences.
        max_string_length = max([len(texts[cur_index+i]) 
            for i in range(0, self.minibatch_size)])
          
        # initialize the arrays
        X_data = np.zeros([self.minibatch_size, max_length, 
            self.feat_dim*self.spectrogram + self.mfcc_dim*(not self.spectrogram)])
#         here we create all blanks for labels :). maybe you say why? because 1 * number_of_blank. (default is 1 * 28, because that code had 27 characters.)
        labels = np.ones([self.minibatch_size, max_string_length]) * 77
        input_length = np.zeros([self.minibatch_size, 1])
        label_length = np.zeros([self.minibatch_size, 1])  
        for i in range(0, self.minibatch_size):
            # calculate X_data & input_length
            feat = features[i]
            input_length[i] = feat.shape[0]
            X_data[i, :feat.shape[0], :] = feat

            # calculate labels & label_length
            label = np.array(text_to_int_sequence(texts[cur_index+i])) 
            labels[i, :len(label)] = label
            label_length[i] = len(label)
 
        # return the arrays
        outputs = {'ctc': np.zeros([self.minibatch_size])}
        inputs = {'the_input': X_data, 
                  'the_labels': labels, 
                  'input_length': input_length, 
                  'label_length': label_length 
                 }
        return (inputs, outputs)

    def shuffle_data_by_partition(self, partition):
        """ Shuffle the training or validation data
        """
        if partition == 'train':
#             self.train_audio_paths, self.train_durations, self.train_texts = self.shuffle_data(
#                 self.train_audio_paths, self.train_durations, self.train_texts)
            self.train_audio_paths, self.train_texts = self.shuffle_data(
                self.train_audio_paths,  self.train_texts)
        elif partition == 'valid':
#             self.valid_audio_paths, self.valid_durations, self.valid_texts = self.shuffle_data(
#                 self.valid_audio_paths, self.valid_durations, self.valid_texts)
            self.valid_audio_paths, self.valid_texts = self.shuffle_data(
                self.valid_audio_paths, self.valid_texts)
        else:
            raise Exception("Invalid partition. "
                "Must be train/validation")

    def sort_data_by_duration(self, partition):
        """ Sort the training or validation sets by (increasing) duration
        """
        if partition == 'train':
            self.train_audio_paths, self.train_durations, self.train_texts = sort_data(
                self.train_audio_paths, self.train_durations, self.train_texts)
        elif partition == 'valid':
            self.valid_audio_paths, self.valid_durations, self.valid_texts = sort_data(
                self.valid_audio_paths, self.valid_durations, self.valid_texts)
        else:
            raise Exception("Invalid partition. "
                "Must be train/validation")

    def next_train(self):
        """ Obtain a batch of training data
        """
        while True:
            ret = self.get_batch('train')
            self.cur_train_index += self.minibatch_size
            if self.cur_train_index >= len(self.train_texts) - self.minibatch_size:
                self.cur_train_index = 0
                self.shuffle_data_by_partition('train')
            yield ret    

    def next_valid(self):
        """ Obtain a batch of validation data
        """
        while True:
            ret = self.get_batch('valid')
            self.cur_valid_index += self.minibatch_size
            if self.cur_valid_index >= len(self.valid_texts) - self.minibatch_size:
                self.cur_valid_index = 0
                self.shuffle_data_by_partition('valid')
            yield ret

    def next_test(self):
        """ Obtain a batch of test data
        """
        while True:
            ret = self.get_batch('test')
            self.cur_test_index += self.minibatch_size
            if self.cur_test_index >= len(self.test_texts) - self.minibatch_size:
                self.cur_test_index = 0
            yield ret

#             this method is called in second Main step 2.0
    def load_train_data(self, desc_file=PATH + "train.tsv"):
        self.load_metadata_from_desc_file(desc_file, 'train')
        self.fit_train()
#         I don't want to test this yet.
        if self.sort_by_duration:
            self.sort_data_by_duration('train')

            # this method is called in third main step 3.0
    def load_validation_data(self, desc_file=PATH + "dev.tsv"):
        self.load_metadata_from_desc_file(desc_file, 'dev')
        #         I don't want to test this yet.
        if self.sort_by_duration:
            self.sort_data_by_duration('valid')

    def load_test_data(self, desc_file=PATH + "test.tsv"):
        self.load_metadata_from_desc_file(desc_file, 'test')
    
#     2.3
    def drop_col(df, cols):
        for col in cols:
            df.drop([col], axis=1, inplace=True)


#             2.2
    def prepare_dataset(df_address):
        df =pd.read_csv(df_address, delimiter='\t',encoding='utf-8')
        drop_col(df, ['age', 'up_votes', 'client_id', 'accent', 'down_votes', 'gender'])
        duration_list = []
        for path in list(df.path):
            y, sr = librosa.load(PATH + 'clips/' +  path, sr=16000) # sound is 
            duration = librosa.core.get_duration(y=y , sr=sr )
            duration_list.append(duration)
        df['durations'] = duration_list
        return df
    
    
#     2.1
    def load_metadata_from_desc_file(self, desc_file, partition):
        """ Read metadata from a tsv file
            (possibly takes long, depending on the filesize)
        Params:
            desc_file (str):  Path to folder which contains all tsv files and clips folder
            partition (str): One of 'train', 'dev' or 'test'
        """
        audio_paths, durations, texts = [], [], []
        # read tsv file
# TEMP NEXT LINE IS BECOMING COMMENT
#         df = prepare_dataset(desc_file)

        if partition == 'train':
            df = trainn
            self.train_audio_paths = list(df.path)
#             self.train_durations = list(df.durations)
            self.train_texts = list(df.sentence)
        elif partition == 'dev':
            df = devv
            self.valid_audio_paths = list(df.path)
#             self.valid_durations = list(df.durations)
            self.valid_texts = list(df.sentence)
        elif partition == 'test':
            self.test_audio_paths = df.path
#             self.test_durations = df.durations
            self.test_texts = df.sentence
        else:
            raise Exception("Invalid partition to load metadata. "
             "Must be train/validation/test")
            
    
    #2.4
    def fit_train(self, k_samples=100):
        """ Estimate the mean and std of the features from the training set
        Params:
            k_samples (int): Use this number of samples for estimation
        """
        k_samples = min(k_samples, len(self.train_audio_paths))
        samples = self.rng.sample(self.train_audio_paths, k_samples)
        feats = [self.featurize(s) for s in samples]
        feats = np.vstack(feats)
        self.feats_mean = np.mean(feats, axis=0)
        self.feats_std = np.std(feats, axis=0)
        
    def featurize(self, audio_clip):
        """ For a given audio clip, calculate the corresponding feature
        Params:
            audio_clip (str): Path to the audio clip
        """
        if self.spectrogram:
            return spectrogram_from_file(
                audio_clip, step=self.step, window=self.window,
                max_freq=self.max_freq)
        else:
#             I need t change this with librosa loading files.
            audio, sample_rate = librosa.load(PATH + 'clips/' + audio_clip, sr=16000)
#             audio = padarray(audio, 160000)
#             (rate, sig) = wav.read(audio_clip)
            return mfcc(audio, sample_rate, numcep=self.mfcc_dim)

    def normalize(self, feature, eps=1e-14):
        """ Center a feature using the mean and std
        Params:
            feature (numpy.ndarray): Feature to normalize
        """
        return (feature - self.feats_mean) / (self.feats_std + eps)

#     def shuffle_data(self, a, d, t):
    def shuffle_data(self, a, t):
        """ Shuffle the data (called after making a complete pass through 
            training or validation data during the training process)
        Params:
            audio_paths (list): Paths to audio clips
            durations (list): Durations of utterances for each audio clip
            texts (list): Sentences uttered in each audio clip
        """
        p = np.random.permutation(len(a))
        a = [a[i] for i in p]
#         d = [d[i] for i in p]
        t = [t[i] for i in p]
#         return a, d, t
        return a, t


    def sort_data(audio_paths, durations, texts):
        """ Sort the data by duration 
        Params:
            audio_paths (list): Paths to audio clips
            durations (list): Durations of utterances for each audio clip
            texts (list): Sentences uttered in each audio clip
        """
        p = np.argsort(durations).tolist()
        audio_paths = [audio_paths[i] for i in p]
        durations = [durations[i] for i in p] 
        texts = [texts[i] for i in p]
        return audio_paths, durations, texts

    def vis_train_features(index=0):
        """ Visualizing the data point in the training set at the supplied index
        """
        # obtain spectrogram
        audio_gen = AudioGenerator(spectrogram=True)
        audio_gen.load_train_data()
        vis_audio_path = audio_gen.train_audio_paths[index]
        vis_spectrogram_feature = audio_gen.normalize(audio_gen.featurize(vis_audio_path))
        # obtain mfcc
        audio_gen = AudioGenerator(spectrogram=False)
        audio_gen.load_train_data()
        vis_mfcc_feature = audio_gen.normalize(audio_gen.featurize(vis_audio_path))
        # obtain text label
        vis_text = audio_gen.train_texts[index]
        # obtain raw audio
        vis_raw_audio, _ = librosa.load(vis_audio_path)
        # print total number of training examples
        print('There are %d total training examples.' % len(audio_gen.train_audio_paths))
        # return labels for plotting
        return vis_text, vis_raw_audio, vis_mfcc_feature, vis_spectrogram_feature, vis_audio_path


    def plot_raw_audio(vis_raw_audio):
        # plot the raw audio signal
        fig = plt.figure(figsize=(12,3))
        ax = fig.add_subplot(111)
        steps = len(vis_raw_audio)
        ax.plot(np.linspace(1, steps, steps), vis_raw_audio)
        plt.title('Audio Signal')
        plt.xlabel('Time')
        plt.ylabel('Amplitude')
        plt.show()

    def plot_mfcc_feature(vis_mfcc_feature):
        # plot the MFCC feature
        fig = plt.figure(figsize=(12,5))
        ax = fig.add_subplot(111)
        im = ax.imshow(vis_mfcc_feature, cmap=plt.cm.jet, aspect='auto')
        plt.title('Normalized MFCC')
        plt.ylabel('Time')
        plt.xlabel('MFCC Coefficient')
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="5%", pad=0.05)
        plt.colorbar(im, cax=cax)
        ax.set_xticks(np.arange(0, 26, 2), minor=False);
        plt.show()

    def plot_spectrogram_feature(vis_spectrogram_feature):
        # plot the normalized spectrogram
        fig = plt.figure(figsize=(12,5))
        ax = fig.add_subplot(111)
        im = ax.imshow(vis_spectrogram_feature, cmap=plt.cm.jet, aspect='auto')
        plt.title('Normalized Spectrogram')
        plt.ylabel('Time')
        plt.xlabel('Frequency')
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="5%", pad=0.05)
        plt.colorbar(im, cax=cax)
        plt.show()


# Functions to train our model. 

In [ ]:
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

def add_ctc_loss(input_to_softmax):
    the_labels = Input(name='the_labels', shape=(None,), dtype='float32')
    input_lengths = Input(name='input_length', shape=(1,), dtype='int64')
    label_lengths = Input(name='label_length', shape=(1,), dtype='int64')
    output_lengths = Lambda(input_to_softmax.output_length)(input_lengths)
    # CTC loss is implemented in a lambda layer
    loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')(
        [input_to_softmax.output, the_labels, output_lengths, label_lengths])
    model = Model(
        inputs=[input_to_softmax.input, the_labels, input_lengths, label_lengths], 
        outputs=loss_out)
    return model

def train_model(input_to_softmax, 
                pickle_path,
                save_model_path,
                train_json='train_corpus.json',
                valid_json='valid_corpus.json',
                minibatch_size=20,
                spectrogram=True,
                mfcc_dim=26,
                optimizer=SGD(lr=0.02, decay=1e-6, momentum=0.9, nesterov=True, clipnorm=5),
                epochs=20,
                verbose=1,
                sort_by_duration=False,
                max_duration=10.0):
    
    # create a class instance for obtaining batches of data
    audio_gen = AudioGenerator(minibatch_size=minibatch_size, 
        spectrogram=spectrogram, mfcc_dim=mfcc_dim, max_duration=max_duration,
        sort_by_duration=sort_by_duration)
    # add the training data to the generator
    audio_gen.load_train_data()
    audio_gen.load_validation_data()
    # calculate steps_per_epoch
    num_train_examples=len(audio_gen.train_audio_paths)
    steps_per_epoch = num_train_examples//minibatch_size
    # calculate validation_steps
    num_valid_samples = len(audio_gen.valid_audio_paths) 
    validation_steps = num_valid_samples//minibatch_size
    
    # add CTC loss to the NN specified in input_to_softmax
    model = add_ctc_loss(input_to_softmax)

    # CTC loss is implemented elsewhere, so use a dummy lambda function for the loss
    model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=optimizer)

    # make results/ directory, if necessary
    if not os.path.exists('results'):
        os.makedirs('results')

    # add checkpointer
#     checkpointer = ModelCheckpoint(filepath='results/'+save_model_path, verbose=0)

    # train the model
    hist = model.fit_generator(generator=audio_gen.next_train(), steps_per_epoch=steps_per_epoch,
        epochs=epochs, validation_data=audio_gen.next_valid(), validation_steps=validation_steps, verbose=verbose)

    # save model loss
    with open('results/'+pickle_path, 'wb') as f:
        pickle.dump(hist.history, f)

# OKAY
### until here we created every thing we need to train our model.
now you just need to set your model structure and send it to our Magical function

#### Very simple RNN model for test

In [ ]:
# output dimesion is so important. be tedade kalame hayi ke addad kardam + blank
def simple_rnn_model(input_dim, output_dim=78):
    """ Build a recurrent network for speech 
    """
    # Main acoustic input
    input_data = Input(name='the_input', shape=(None, input_dim))
    # Add recurrent layer
    simp_rnn = GRU(output_dim, return_sequences=True, 
                 implementation=2, name='rnn')(input_data)
    # Add softmax activation layer
    y_pred = Activation('softmax', name='softmax')(simp_rnn)
    # Specify the model
    model = Model(inputs=input_data, outputs=y_pred)
    model.output_length = lambda x: x
    print(model.summary())
    return model

In [ ]:
# Change to 26 if you would like to use MFCC features
model_0 = simple_rnn_model(input_dim=161)

In [ ]:
train_model(input_to_softmax=model_0,
            pickle_path='model_0.pickle',
            save_model_path='model_0.h5',
            spectrogram=True) # Change to False if you would like to use MFCC features